In [1]:
import torch
import os, sys
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.1+cu102 (GeForce RTX 2060)


In [2]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
mult = 1024 * 1024
print(t / mult, r / mult, a / mult, f / mult)

5934.5625 0.0 0.0 0.0


In [3]:
%%markdown
## Dataset labels remapping for COCO 80 classes

## Dataset labels remapping for COCO 80 classes


In [23]:
old_names = ['bird', 'cat', 'dog', 'person']
new_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [25]:
remap_dict = dict([(str(i), str(new_names.index(label))) for i, label in enumerate(old_names)])
remap_dict

{'0': '14', '1': '15', '2': '16', '3': '0'}

In [26]:
dataset_labels_location = '../datasets/hack_scratch/train/labels/'

In [27]:
label_files = os.listdir(dataset_labels_location)
label_files = [os.path.join(dataset_labels_location, lf) for lf in label_files]
label_files[:5]

['../datasets/hack_scratch/train/labels/8952_6032_jpg.rf.a41ec8adfb776687f3f256f5c9bf9b07.txt',
 '../datasets/hack_scratch/train/labels/9548_529_jpg.rf.1ad2c9b20d9ecff36e2cb197c3d624d1.txt',
 '../datasets/hack_scratch/train/labels/6520_2401_jpg.rf.cfac8d0d1ed5d9f7ed2e1e83f0905259.txt',
 '../datasets/hack_scratch/train/labels/8979_6763_jpg.rf.631f5d9e207bd357d2fcff3d4b1cc2e7.txt',
 '../datasets/hack_scratch/train/labels/-----------_2021-10-20_160826_png.rf.f1c2636bd5dc47a781f4ddeeffca96bc.txt']

In [8]:
# for lf in label_files:
#     fixed_lines = []
#     with open(lf, 'r', encoding="utf-8") as file:
#         all_lines = file.readlines()
#         for line in all_lines:
#             splitted = line.split(' ')
#             splitted[0] = remap_dict[splitted[0]]
#             fixed_lines.append(" ".join(splitted))
#     with open(lf, 'w', encoding="utf-8") as file:
#         file.writelines(fixed_lines)

In [28]:
# for lf in label_files:
#     fixed_lines = []
#     with open(lf, 'r', encoding="utf-8") as file:
#         all_lines = file.readlines()
#         for line in all_lines:
#             splitted = line.split(' ')
#             if splitted[0] == '14':
#                 continue
#             if splitted[0] == '15':
#                 splitted[0] = '0'
#             if splitted[0] == '16':
#                 splitted[0] = '1'
#             if splitted[0] == '0':
#                 splitted[0] = '2'
#             fixed_lines.append(" ".join(splitted))
#     with open(lf, 'w', encoding="utf-8") as file:
#         file.writelines(fixed_lines)

In [29]:
# for lf in label_files:
#     with open(lf, 'r', encoding="utf-8") as file:
#         print(lf, '\n', file.readlines())

../datasets/hack_scratch/train/labels/8952_6032_jpg.rf.a41ec8adfb776687f3f256f5c9bf9b07.txt 
 ['2 0.437890625 0.6307397959183674 0.02734375 0.11607142857142858\n', '1 0.66328125 0.6326530612244898 0.03125 0.03316326530612245']
../datasets/hack_scratch/train/labels/9548_529_jpg.rf.1ad2c9b20d9ecff36e2cb197c3d624d1.txt 
 ['2 0.31953125 0.5459183673469388 0.0171875 0.07653061224489796\n', '1 0.101953125 0.6358418367346939 0.03046875 0.039540816326530615']
../datasets/hack_scratch/train/labels/6520_2401_jpg.rf.cfac8d0d1ed5d9f7ed2e1e83f0905259.txt 
 ['1 0.720703125 0.6288265306122449 0.03203125 0.03826530612244898']
../datasets/hack_scratch/train/labels/8979_6763_jpg.rf.631f5d9e207bd357d2fcff3d4b1cc2e7.txt 
 ['1 0.9535126324368378 0.8584183673469388 0.05786471067644662 0.13520408163265307']
../datasets/hack_scratch/train/labels/-----------_2021-10-20_160826_png.rf.f1c2636bd5dc47a781f4ddeeffca96bc.txt 
 ['1 0.4876190476190476 0.49521531100478466 0.6495238095238095 0.9776714513556619\n', '2 0.

In [10]:
%%markdown
## Train stage

## Train stage


In [11]:
# %load_ext tensorboard
# %tensorboard --logdir runs/train

In [12]:
%%markdown
### Проверка базовой модели yolov5l на датасете

### Проверка базовой модели yolov5l на датасете


In [16]:
!python val.py --weights yolov5l.pt --verbose --data coco_finetune.yaml --img 640 --batch-size 32 --conf-thres 0.05 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/lct_hack/yolov5/data/coco_finetune.yaml, weights=['yolov5l.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.45, task=val, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-13-gfc36064 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5934.5625MB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
val: Scanning '../datasets/coco_finetune/train/labels' images and labels...698 f
val: New cache created: ../datasets/coco_finetune/train/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        698       1187      0.565       0.46      0.532      0.365
              person        698        440      0.723      0.857      0.859       0.51
                bird        698         26      0.194      0.577      0.198      0

In [17]:
%%markdown
### Файн-тьюнинг на датасете

### Файн-тьюнинг на датасете


In [16]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 600 --batch 8 --hyp hyp.finetune.yaml --epochs 8 --data coco_finetune.yaml --weights yolov5l.pt --project "projects/COCO-street-finetune-6" --cache

train: weights=yolov5l.pt, cfg=, data=coco_finetune.yaml, hyp=hyp.finetune.yaml, epochs=8, batch_size=8, imgsz=600, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=projects/COCO-street-finetune-6, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 12 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-13-gfc36064 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5934.5625MB)

hyperparameters: lr0=0.0032, lrf=0.12, momentum=0.843, weight_decay=0.00036, warmup_epochs=2.0, warmup_momentum=0.5, warmup_bias_lr=0.05, box=0.0296, cls=0.243, cls_pw=0.631, obj=0.301, obj_pw=0.911, iou_t=0.2, anch

Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        698       1187      0.703      0.702      0.763      0.496
              person        698        440      0.708      0.843      0.865      0.511
                bird        698         26      0.842      0.462      0.692      0.403
                 cat        698        122      0.554      0.738      0.728      0.586
                 dog        698        599      0.708      0.766      0.766      0.485
Results saved to projects/COCO-street-finetune-6/exp


In [17]:
%%markdown
### Полноценное обучение с нуля

### Полноценное обучение с нуля


In [1]:
!python train.py --img 600 --batch 8 --epochs 17 --data hack_scratch.yaml --weights yolov5l.pt --project "projects/COCO-street-from-scratch-2" --cache

train: weights=yolov5l.pt, cfg=, data=hack_scratch.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=17, batch_size=8, imgsz=600, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=projects/COCO-street-from-scratch-2, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 12 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-13-gfc36064 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5934.5625MB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        698       1162      0.698      0.681      0.729      0.301

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      9/16     4.45G   0.05004   0.01136  0.009075         3       608: 100%|█| 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        698       1162      0.733      0.707      0.791      0.398

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     10/16     4.45G   0.04628    0.0108  0.008088         5       608: 100%|█| 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        698       1162        0.8      0.753      0.826      0.382

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     11/16     4.45G   0.04428   0.01111  0.007505         6       608: 100%|█| 
               Class     Images     L

In [7]:
%%markdown
### Проверки на COCO val датасете

### Проверки на COCO val датасете


In [14]:
# Run YOLOv5x on COCO val2017
!python val.py --weights yolov5l.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.05 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/lct_hack/yolov5/data/coco.yaml, weights=['yolov5l.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.45, task=val, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-13-gfc36064 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5934.5625MB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
val: Scanning '../datasets/coco/val2017.cache' images and labels... 4952 found, 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       5000      36335      0.754      0.605      0.683      0.508
              person       5000      10777      0.841      0.743      0.837      0.623
             bicycle       5000        314      0.781      0.567      0.674      0.426
                 car       5000       1918      0.776      0.665      0.74

DONE (t=2.60s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.471
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.654
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.511
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.303
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.524
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.603
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.360
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.556
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.575
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.386
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.628
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.723
Results saved to runs/va

In [15]:
# Run YOLOv5x on COCO val2017
!python val.py --weights projects/COCO-street-finetune-5/exp/weights/best.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.1 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/lct_hack/yolov5/data/coco.yaml, weights=['projects/COCO-street-finetune-5/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.1, iou_thres=0.45, task=val, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-13-gfc36064 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5934.5625MB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
val: Scanning '../datasets/coco/val2017.cache' images and labels... 4952 found, 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       5000      36335      0.707      0.561      0.651       0.48
              person       5000      10777      0.801      0.736      0.819      0.594
             bicycle       5000        314       0.78      0.475      0.624      0.411
                 car

DONE (t=12.21s).
Accumulating evaluation results...
DONE (t=1.72s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.554
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.427
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.227
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.443
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.546
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.315
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.459
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.467
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.265
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.509
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

In [17]:
# Run YOLOv5x on COCO val2017
!python val.py --weights projects/COCO-street-finetune-6/exp/weights/best.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.1 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/lct_hack/yolov5/data/coco.yaml, weights=['projects/COCO-street-finetune-6/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.1, iou_thres=0.45, task=val, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-13-gfc36064 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5934.5625MB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
val: Scanning '../datasets/coco/val2017.cache' images and labels... 4952 found, 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       5000      36335      0.684      0.553      0.642      0.477
              person       5000      10777      0.753      0.761      0.815      0.609
             bicycle       5000        314      0.754      0.459      0.634      0.434
                 car

DONE (t=11.12s).
Accumulating evaluation results...
DONE (t=1.71s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.503
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.400
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.521
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.295
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.416
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.210
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.456
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

In [ ]:
# Run YOLOv5x on COCO val2017
!python val.py --weights projects/COCO-street-finetune-6/exp/weights/best.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.1 --iou-thres 0.45 --half